<a href="https://colab.research.google.com/github/proyectosRVyderivados/cristina/blob/main/1BonoSintetico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install numpy-financial

#Creación de un bono sintético cupón cero
Supongamos dos bonos A y B, ambos a 5 años con las siguientes características:

* Bono A
  * Nominal: 10.000 €
  * Cupón anual: 2%
* Bono B
  * Nominal: 4.000 €
  * Cupón anual: 25%

La ETTI para los diferentes años es:
* año 1: 2%
* año 2: 4%
* año 3: 6%
* año 4: 8%
* año 5: 10%

Se pide:

* Calcular el precio de ambos bonos
* Calcular la TIR de ambos bonos
* Crear el bono C que es un bono sintético que se forma combinando los bonos A y B para conseguir un bono cupón cero a 5 años.
* Calcular la TIR del bono C y comprobar que es igual a la ETTI a 5 años, que es del 10%.

In [ ]:
import numpy_financial as npf
# DATOS BONO A
nominalA = 10_000
cuponA = 0.02
n = 5 # años, para A y B
# DATOS BONO B
nominalB = 4_000
cuponB = 0.25
# ETTI
etti =[.02,.04,.06,.08,.10]
# FLUJOS DE CAJA DE LOS BONOS
def flujos(nominal, cupon, n):
    cf = [0]  # vector con el cash flow
    for i in range(1, n+1):
        if i < n:
            cf.append(cupon * nominal)
        elif i == n:
            cf.append(cupon * nominal + nominal)
    return cf

# Flujos de caja de los Bonos A y B
flujosA = flujos(nominalA, cuponA, n)
flujosB = flujos(nominalB, cuponB, n)

 # PRECIO DE UN BONO CON LA ETTI
def precio_etti(flujos, etti):
    precio = 0
    for i in range(1, n+1):
        precio += flujos[i] * (1+etti[i-1])**-i
    return precio

# Precios de los Bonos A y B
precioA = precio_etti(flujosA, etti)
precioB = precio_etti(flujosB, etti)
print(f"El precio del bono A es {precioA}")
print(f"El precio del bono B es {precioB}")

# Flujos de caja de los bonos A y B
bonoA = flujosA[:] #copia los valores pero no me modificará el vector de bono
bonoA[0] = -precioA
bonoB = flujosB[:]
bonoB[0] = -precioB

# CREACIÓN DEL BONO SINTÉTICO C
# m es el número de bonos que se han de comprar o vender de uno de los bonos
m = max(cuponA*nominalA, cuponB*nominalB) / min(cuponB*nominalB, cuponA*nominalA)
# con los datos de ejemplo m = 1000 / 200 = 5

# Flujos de caja del bono C
bonoC = [0]*(n+1)
for i in range(0, n+1):
    bonoC[i] = m * bonoA[i] - bonoB[i]
print(f"Flujos de caja del bono C: {bonoC}")
precioC=-bonoC[0]
print(f"Precio del bono c: {precioC: .6}")
# Calcular la TIR de los bonos
tirA = npf.irr(bonoA)
tirB = npf.irr(bonoB)
tirC = npf.irr(bonoC)
print(f"La TIR del bono A es: {tirA:.6%}")
print(f"La TIR del bono B es: {tirB:.6%}")
print(f"La TIR del bono C es: {tirC:.6%}")

El precio del bono A es 7029.316996345231
El precio del bono B es 6584.2041210050265
Flujos de caja del bono C: [-28562.38086072113, 0.0, 0.0, 0.0, 0.0, 46000.0]
Precio del bono c:  28562.4
La TIR del bono A es: 9.795736%
La TIR del bono B es: 8.573790%
La TIR del bono C es: 10.000000%


#Reinversión de flujos de caja intermedios
La importancia de trabajar con bonos cupón cero.

La TIR es la Tasa Interna de Rentabilidad, la palabra 'Interna' indica que si la operación financiera analizada se mezcla con otras operaciones la rentabilidad prometida por la TIR se podría alterar.

Una operación de inversión que tenga flujos de caja intermedios, por ejemplo un bono cupón explícito, puede no proporcionar a su propietario la rentabilidad que promete la TIR del bono si el inversor no se preocupa de reinvertir los flujos de caja intermedios hasta el final de la operación.

Supongamos un bono cupón explícito con las siguientes características:

* Nominal 10.000 €
* Cupón 10%
* Madura a los 10 años (n=10)
* Precio de adquisición 10.000 €

Se pide:

* Calcular la TIR del bono
* Calcular la rentabilidad del inversor supueto que reinvierta los flujos de caja intermedios hasta el momento de vencimiento, a las siguientes rentabilidades:
  * al 0%
  * al 10%
  * al 20%
  
Crear un bono cupón cero, al mismo plazo, por el mismo precio, cuyo último flujo de caja sea el necesario para proporcionar una rentabilidad del 10% y comprobar que su TIR es de 10%.

In [ ]:
import numpy_financial as npf

# DATOS
nominal = 10_000
cupon = .1
n = 10 # años
precio = 10_000

# Flujos de caja del bono
cf = [0]*(n+1)
for i in range(1,n+1):
    cf[i] = cupon * nominal
cf[n] += nominal
cf[0] = -precio
print(f"La TIR del bono es: {npf.irr(cf):.2%}")

# Montante de la Reinversión
def montante(cf,r):
    m = 0 # montante
    for i in range(1, n+1):
        m += cf[i]*(1+r)**(n-i)
    return m

tasas_reinversion = [0,.1,.2]
for t in tasas_reinversion:
    m = montante(cf,t)
    print()
    print(f"El montante reinvirtiendo al tanto del {t:.0%} es {m:,.2f} €")
    print(f"La rentabilidad del inversor reinvirtiendo al tanto {t:.0%} es {(m/precio)**(1/n)-1:.2%}")

La TIR del bono es: 10.00%

El montante reinvirtiendo al tanto del 0% es 20,000.00 €
La rentabilidad del inversor reinvirtiendo al tanto 0% es 7.18%

El montante reinvirtiendo al tanto del 10% es 25,937.42 €
La rentabilidad del inversor reinvirtiendo al tanto 10% es 10.00%

El montante reinvirtiendo al tanto del 20% es 35,958.68 €
La rentabilidad del inversor reinvirtiendo al tanto 20% es 13.65%
